In [17]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
# import importlib
# importlib.reload(tf)

In [2]:
data_path = "../../data/cull%i/model_data/" % 1

In [3]:
devtest_aa_dict = np.load(data_path + 'devtest_aa_dict.npy')[()]
devtest_cmap_dict = np.load(data_path + 'devtest_cmap_dict.npy')[()]
train_aa_dict = np.load(data_path + 'train_aa_dict.npy')[()]
train_cmap_dict = np.load(data_path + 'train_cmap_dict.npy')[()]
valid_aa_dict = np.load(data_path + 'valid_aa_dict.npy')[()]
valid_cmap_dict = np.load(data_path + 'valid_cmap_dict.npy')[()]

In [4]:
# devtest_aa_dict

In [5]:
# from keras import backend as K
# from keras.engine.topology import Layer
# # from keras.layers import Layer

# # Need to create my own outer product layer
# class OuterConcat(Layer):
#     def __init__(self, output_dim, **kwargs):
#         self.output_dim = output_dim
#         self.name = "HII"
        
#     def built(self, input_shape):
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight(name='kernel', 
#                                       shape=(input_shape[1], self.output_dim),
#                                       initializer='uniform',
#                                       trainable=True)
#         super(MyLayer, self).build(input_shape)  # Be sure to call this at the end
        
#     def call(self, x):
#         return K.dot(x, self.kernel)
    
#     def compute_output_shape(self, input_shape):
#         return (input_shape[0], self.output_dim)

In [6]:
class MyLayer(tf.keras.layers.Layer):

    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(MyLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(MyLayer, self).build(input_shape)  # Be sure to call this at the end

    def call(self, x):
        return K.dot(x, self.kernel)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)

In [7]:
# seq2pairwise(x)
# x = [1, 2, 3]
# y = [4, 5, 6]
# X, Y = tf.meshgrid(x, y)
# X.get_shape

In [8]:

# def seq2pairwise(incoming):
#     L = tf.shape(incoming)[1]
#     #save the indexes of each position
#     v = tf.range(0, L, 1)
#     i, j = tf.meshgrid(v, v)
#     m = (i+j)/2
#     #switch batch dim with L dim to put L at first
#     incoming2 = tf.transpose(incoming, perm=[1, 0, 2])
#     #full matrix i with element in incomming2 indexed i[i][j]
#     out1 = tf.nn.embedding_lookup(incoming2, i)
#     out2 = tf.nn.embedding_lookup(incoming2, j)
#     out3 = tf.nn.embedding_lookup(incoming2, m)
#     #concatante final feature dim together
#     out = tf.concat([out1, out2, out3], axis=3)
#     #return to original dims
#     output = tf.transpose(out, perm=[2, 0, 1, 3])
#     return output

In [9]:
# class MyDenseLayer(tf.keras.layers.Layer):
#     def __init__(self, num_outputs):
#         super(MyDenseLayer, self).__init__()
#         self.num_outputs = num_outputs

#     def build(self, input_shape):
#         self.kernel = self.add_variable(
#             "kernel",
#             shape=[int(input_shape[-1]),
#                    self.num_outputs])

#     def call(self, input):
#         return tf.matmul(input, self.kernel)


# layer = MyDenseLayer(10)
# print(layer)
# print(layer(tf.zeros([10, 5])))
# print(layer.trainable_variables)

Tensor("my_dense_layer/MatMul:0", shape=(10, 10), dtype=float32)
[<tf.Variable 'my_dense_layer/kernel:0' shape=(5, 10) dtype=float32>]


In [10]:
class OuterProduct(tf.keras.layers.Layer):
    """
    Given a layer of size (B, L, N), create 
    a layer of size (B, L, L, 3N).
    For example, if we have 
    
    """
    def __init__(self):
        super(OuterProduct, self).__init__()

    def call(self, incoming):

        L = tf.shape(incoming)[1]
        # save the indexes of each position
        v = tf.range(0, L, 1)

        i, j = tf.meshgrid(v, v)

        m = tf.cast((i+j)/2, tf.int32)

        # switch batch dim with L dim to put L at first
        incoming2 = tf.transpose(incoming, perm=[1, 0, 2])

        # full matrix i with element in incomming2 indexed i[i][j]
        out1 = tf.nn.embedding_lookup(incoming2, i)
        out2 = tf.nn.embedding_lookup(incoming2, j)
        out3 = tf.nn.embedding_lookup(incoming2, m)

        # concatanate final feature dim together
        out = tf.concat([out1, out2, out3], axis=3)
        # return to original dims
        output = tf.transpose(out, perm=[2, 0, 1, 3])
        return output

In [29]:
class ResidualBlock1D(tf.keras.layers.Layer):
    """
    All the layers in the residual block will have the 
    same number of features and stride.
    There are two layers in this block.

    """

    def __init__(self, stride, activation):
        self.stride = stride
        self.activation = activation
        super(ResidualBlock1D, self).__init__()

#     def build(self, input_shape):
#         # Create a trainable weight variable for this layer.
#         self.kernel = self.add_weight(
#             name='kernel',
#             shape=(input_shape[1], self.output_dim),
#             initializer='uniform',
#             trainable=True
#         )
#         # Be sure to call this at the end
#         super(MyLayer, self).build(input_shape)

    def call(self, x):
        size = int(x.shape[-1])
        y = tf.keras.layers.Conv1D(
            size,
            self.stride,
            activation=self.activation,
        )(x)
        y = tf.keras.layers.Conv1D(
            size,
            self.stride,
            activation=self.activation,
        )(y)

        y = tf.keras.layers.add([y, x])

        return y


class ResidualBlock2D(tf.keras.layers.Layer):
    """
    All the layers in the residual block will have the 
    same number of features and stride.
    There are two layers in this block.

    """

    def __init__(self, stride, activation):
        self.stride = stride
        self.activation = activation
        super(ResidualBlock2D, self).__init__()

    def call(self, x):
        size = int(x.shape[-1])
        y = tf.keras.layers.Conv2D(
            size,
            self.stride,
            activation=self.activation,
            padding='same'
        )(x)
        y = tf.keras.layers.Conv2D(
            size,
            self.stride,
            activation=self.activation,
            padding='same'
        )(y)

        y = tf.keras.layers.add([y, x])

        return y

In [73]:
# def conv_block(x, size, stride, conv_dim, num_layers, activation, padding):
#     if conv_dim == 1:
#         for i in range(num_layers):
#             x = tf.keras.layers.Conv1D(
#                 size,
#                 stride,
#                 activation=activation,
#                 padding=padding
#             )(x)
#             x = tf.keras.layers.BatchNormalization()(x)
#         return x

#     elif conv_dim == 2:
#         for i in range(num_layers):
#             x = tf.keras.layers.Conv2D(
#                 size,
#                 stride,
#                 activation=activation,
#                 padding=padding
#             )(x)
#             x = tf.keras.layers.BatchNormalization()(x)

#         return x
#     else:
#         raise ValueError("Not an available convnet dimension")


def residual_conv_block(x, convnet, stride, num_layers=2, activation="relu", padding="same"):

    size = int(x.shape[-1])
    y = x
    
    if convnet == "1d convnet":
        for i in range(num_layers):
            y = tf.keras.layers.Conv1D(
                size,
                stride,
                activation=activation,
                padding=padding
            )(y)
            y = tf.keras.layers.BatchNormalization()(y)
    elif convnet == "2d convnet":
        for i in range(num_layers):
            y = tf.keras.layers.Conv2D(
                size,
                stride,
                activation=activation,
                padding=padding
            )(y)
            y = tf.keras.layers.BatchNormalization()(y)
    
    else:
        raise ValueError("Not an available convnet dimension")
        
    y = tf.keras.layers.add([y, x])
    return y


# def Residual_block_2d(x, stride, num_layers=2, activation='relu', padding='same'):

#     size = int(x.shape[-1])

# #     y = conv_block(x, size, stride, conv)

#     y = conv_block(x, size, stride, 2, num_layers, activation, padding)
#     y = tf.keras.layers.add([y, x])

#     return y


# def Residual_block_1d(x, stride, num_layers=2, activation='relu', padding='same'):

#     size = int(x.shape[-1])

# #     y = conv_block(x, size, stride, conv)

#     y = conv_block(x, size, stride, 1, num_layers, activation, padding)
#     y = tf.keras.layers.add([y, x])

#     return y

In [68]:
# tf.keras.layers.Conv1D()
x = 1
y = x

y = y + 1
x

1

In [75]:
# from keras.models import Sequential

# model = Sequential()

# from keras import Input

# from keras.models import Model
# from tensorflow.keras.model import Model
# from keras import layers
# from tf.keras.layers
def create_architecture():
    input_tensor = tf.keras.Input(shape=(None, 20))
    x = tf.keras.layers.Conv1D(
        40,
        1,
        activation='relu',
    )(input_tensor)

    x = tf.keras.layers.Conv1D(
        60,
        1,
        activation='relu',
    )(x)

#     x = ResidualBlock1D(1, "relu")(x)


#     x = Residual_block_1d(x, 1)
#     residual_conv_block(x, convnet, stride, num_layers=2, activation="relu", padding="same")
    x = residual_conv_block(x, "1d convnet", 1)

    """
    EDIT: did not have to do this lol.
    Keras 2.2.4

    To make this work, I had to go to:
    /Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/keras/engine/base_layer.py

    and change on line: 1749
    layer.outbound_nodes.append(self)

    to

    layer._outbound_nodes.append(self)
    """

    x = OuterProduct()(x)
#     x = ResidualBlock2D(1, "relu")(x)
#     x = Residual_block_2d(x, 1)
    # x = tf.keras.layers.Dense(128, activation='relu')(x)
    x = residual_conv_block(x, "2d convnet", 1)


    x = tf.keras.layers.Conv2D(2, 1, activation='relu', padding='same')(x)
    # x = tf.keras.layers.Flatten()(x)
    model = tf.keras.models.Model(
        input_tensor,
        x
    )
    
    return model

# x = MyDenseLayer(10)(x)

# third column is the number of feature maps
model = create_architecture()
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_33 (InputLayer)           (None, None, 20)     0                                            
__________________________________________________________________________________________________
conv1d_88 (Conv1D)              (None, None, 40)     840         input_33[0][0]                   
__________________________________________________________________________________________________
conv1d_89 (Conv1D)              (None, None, 60)     2460        conv1d_88[0][0]                  
__________________________________________________________________________________________________
conv1d_90 (Conv1D)              (None, None, 60)     3660        conv1d_89[0][0]                  
__________________________________________________________________________________________________
batch_norm

In [14]:
import random
def aa_generator(x, y):
    """
    
    """
    
    inputs = x.copy()
    outputs = y.copy()
    keys = set (x.keys())
#     print(keys)
    while True:
        try:
            key = random.sample(keys, 1)[0]
            keys.remove(key)
            
            one_hot_aa = x[key]
            one_hot_aa = np.reshape(one_hot_aa, (1,) + one_hot_aa.shape)
            cmap = y[key]
            cmap = np.reshape(cmap, (1,) + cmap.shape + (1,))
            yield one_hot_aa, cmap

#             yield key
        except ValueError:
            # if out of keys, reinsert back the keys
            inputs = x.copy()
            outputs = y.copy()
            keys = set (x.keys())

In [44]:

model.compile(
    optimizer=tf.train.AdamOptimizer(0.001),
    loss="sparse_categorical_crossentropy",
    sample_weight_mode="temporal"
)
model.fit_generator(
    aa_generator(train_aa_dict, train_cmap_dict),
    steps_per_epoch=10, 
    epochs=2
)

/Users/jinli/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/2
10/10 [==============================] - 15s 2s/step - loss: 0.7847
Epoch 2/2
10/10 [==============================] - 19s 2s/step - loss: 0.6932


In [ ]:
test = OuterProduct()
test1 = test(tf.zeros([4, 5, 6]))

In [76]:
type(tf.zeros([4, 5, 6]))

tensorflow.python.framework.ops.Tensor

In [ ]:
# int (x.shape[-1])

# output and input from preprocessing probably doesn't align
import math
math.sqrt(24649)
math.sqrt(24336)

In [ ]:
# test1.shape
tf.VERSION

In [ ]:
zeros = tf.zeros([10, 5, 6])
type(test1)

In [ ]:
# model1 = tf.keras.models.Sequential([
#     tf.keras.layers.Flatten(input_shape=(28, 28)),
#     tf.keras.layers.Dense(128, activation='relu'),
#     tf.keras.layers.Dropout(0.2),
#     tf.keras.layers.Dense(10, activation='softmax'),
# #     MyDenseLayer(10),
# #     test,
# ])

# # model2 = tf.keras.models.Sequential([
# #     test()
# # ])

In [ ]:
# model2.summary()
model1.compile(
    optimizer=tf.train.AdamOptimizer(0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
1 + 3 / 2

In [ ]:
# from keras import models
# from keras import layers
# network = tf.keras.models.Sequential()
# network.add(tf.keras.layers.Dense(512, activation='relu', input_shape=(28 * 28,)))
# network.add(tf.keras.layers.Dense(10, activation='softmax'))

# network.compile(optimizer='rmsprop',
#                 loss='categorical_crossentropy',
#                 metrics=['accuracy'])

In [ ]:
# import tensorflow as tf
# from tensorflow.keras import layers

# model = tf.keras.Sequential([
#     # Adds a densely-connected layer with 64 units to the model:
#     layers.Dense(64, activation='relu', input_shape=(32,)),
#     # Add another:
#     layers.Dense(64, activation='relu'),
#     # Add a softmax layer with 10 output units:
#     layers.Dense(10, activation='softmax')])

# model.compile(
#     optimizer=tf.train.AdamOptimizer(0.001),
#     loss='categorical_crossentropy',
#     metrics=['accuracy']
# )



In [ ]:
i = 0
keys = []
for item in aa_generator(train_aa_dict, train_cmap_dict):
    if i == 142:
        break
    keys.append(item)
    i += 1

In [ ]:
# train_aa_dict['1']
# keys
# len (aa_generator)
len (set (keys))

In [ ]:
# len (train_aa_dict.keys())
array = train_aa_dict['16vp']#.reshape(-1)
np.reshape(array, (1,) + array.shape).shape

In [39]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(
            filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)




input_tensor = tf.keras.Input(shape=(2, 3, 3))
block = ResnetIdentityBlock(1, [1, 2, 3])
nextz = block(input_tensor)
print(block(tf.zeros([1, 2, 3, 3])))
print([x.name for x in block.trainable_variables])

model = tf.keras.models.Model(
    input_tensor,
    nextz
)
model.summary()

Tensor("resnet_identity_block_8_1/Relu_2:0", shape=(1, 2, 3, 3), dtype=float32)
['resnet_identity_block_8/conv2d_39/kernel:0', 'resnet_identity_block_8/conv2d_39/bias:0', 'resnet_identity_block_8/batch_normalization_24/gamma:0', 'resnet_identity_block_8/batch_normalization_24/beta:0', 'resnet_identity_block_8/conv2d_40/kernel:0', 'resnet_identity_block_8/conv2d_40/bias:0', 'resnet_identity_block_8/batch_normalization_25/gamma:0', 'resnet_identity_block_8/batch_normalization_25/beta:0', 'resnet_identity_block_8/conv2d_41/kernel:0', 'resnet_identity_block_8/conv2d_41/bias:0', 'resnet_identity_block_8/batch_normalization_26/gamma:0', 'resnet_identity_block_8/batch_normalization_26/beta:0']
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 2, 3, 3)           0         
_________________________________________________________________
resnet_identity_block_8 (Res (None, 2, 3